![](https://storage.googleapis.com/mle-courses-prod/users/61b6fa1ba83a7e37c8309756/private-files/35ddffa0-70a9-11ef-9b72-9db6eacc12d1-RAGGraph___Neo4J___CamelAI__7_.png)

Tác giả: [ProtonX Team](https://protonx.io/courses/669f5abb02b79700125c9f32)

In [ ]:
!pip install -qq scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 556.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.2 MB/s eta 0:00:00


Tải dữ liệu danh sách các đường dẫn của trang web

In [ ]:
import requests
import xml.etree.ElementTree as ET
import json

# URLs of the sitemaps
sitemap_urls = [
    'https://hoatuoimymy.com/product-sitemap1.xml',
    'https://hoatuoimymy.com/product-sitemap2.xml'
]

all_urls = []

# Function to fetch and parse XML
def fetch_sitemap(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        root = ET.fromstring(response.content)
        # Extract all <loc> elements that contain the URLs
        for url_element in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
            all_urls.append(url_element.text)
    except Exception as e:
        print(f"Error fetching or parsing {url}: {e}")

# Fetch and parse both sitemaps
for sitemap_url in sitemap_urls:
    fetch_sitemap(sitemap_url)

# Write the URLs to a JSON file
with open('all_urls.json', 'w') as f:
    json.dump(all_urls, f, indent=4)

print(f"Extracted {len(all_urls)} URLs and saved to all_urls.json")


Extracted 306 URLs and saved to all_urls.json


Danh sách các trang web

In [ ]:
import pandas as pd
import json

# Specify the filename
filename = '/content/all_urls.json'

# Load the all_urls list from the JSON file
with open(filename, 'r') as file:
    all_urls = json.load(file)

print(f"All URLs loaded from {filename}")


import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup

class CustomSpider(scrapy.Spider):
    name = 'custom_spider'
    start_urls = all_urls[100:]

    # Initialize a counter
    request_count = 0

    def parse(self, response):
        self.request_count += 1  # Increment the counter with each request
        description = ""

        # Scraping the review title (h1 tag inside div.product_title)
        review_title = response.css('h1.product-title::text').get()

        if review_title:
            h1_tag = review_title.strip()
        else:
            h1_tag = ""

        # Now, h1_tag contains the content of the h1 tag
        print(h1_tag)


        price = response.css('span.woocommerce-Price-amount')

        if price:
            price = price.get().strip()
        else:
            price = ""

        # Scraping the ck-content
        ck_contents = response.css('div.woocommerce-Tabs-panel--description')

        for ck_content in ck_contents:
            for element in ck_content.xpath('./*'):
                # Extract the text from h2 and h3 tags
                if element.root.tag == 'h2':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"
                elif element.root.tag == 'h3':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"

                # Extract the text from p tags
                elif element.root.tag == 'p':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"

                # Extract the list items from ul tags
                elif element.root.tag == 'ul':
                    li_tags = element.css('li')
                    for li_tag in li_tags:
                        description += f"- {' '.join(li_tag.css('::text').getall()).strip()}" + "\n"

        # Initialize an empty array to hold image URLs
        image_urls = []

        # Select all the div elements with the specific class
        image_elements = response.css('div.woocommerce-product-gallery__image')

        # Loop through each image element to extract the URLs
        for element in image_elements:
            # Extract the main image URL from the 'data-large_image' attribute
            image_url = element.css('img::attr(data-large_image)').get()

            # Add the extracted image URL to the array
            if image_url:
                image_urls.append(image_url)

        data = {}

        if h1_tag and description:
            data = {
                "url": response.url,  # Add the URL of the request
                "content": description,
                "price": price,
                "title": h1_tag  # h1_tag is now guaranteed to be a string
            }

            yield data
        # Print out the current request count
        print('====> h1_tag', h1_tag)
        print('====>description', description)
        print('====>image_urls', image_urls)
        print('====>price', price)

        self.logger.info(f"Number of requests done: {self.request_count}")
        self.logger.info(f"Crawled: {response.url}")


# Initialize the Scrapy crawler process
process = CrawlerProcess({
    'LOG_LEVEL': 'INFO',
    'FEEDS': {
        'output.json': {
            'format': 'json',
            'encoding': 'utf8',
            'store_empty': False,
            'fields': None,
            'indent': 4,
        },
    },
    'CLOSESPIDER_TIMEOUT': 60000000000,  # Close the spider after 60 seconds (adjust as needed)
    'DOWNLOAD_DELAY': 3,  # Delay of 2 seconds between each request
})

# Start the spider
process.crawl(CustomSpider)
process.start()


All URLs loaded from /content/all_urls.json


INFO:scrapy.utils.log:Scrapy 2.11.2 started (bot: scrapybot)
2024-09-10 09:29:51 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.1 4 Jun 2024), cryptography 43.0.0, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-09-10 09:29:51 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.1 4 Jun 2024), cryptography 43.0.0, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2024-09-10 09:29:51 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRI

Giỏ Hoa M179
====> h1_tag Giỏ Hoa M179
====>description Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:
- Hoa tươi mới được nhập về trong ngày
- Cập nhật xu hướng hoa mới nhất trên thị trường
- Các thiết kế hoa độc lạ và cực kỳ bắt mắt
- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận
- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi
- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách
- Hoàn trả tiền khi khách hàng không hài lòng
- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop
Giỏ Hoa M179
Giỏ Hoa Hồng Đỏ Đẹp: Biểu Tượng Của Tình Yêu Và Sự Trân Trọng
Hoa hồng đỏ luôn được biết đến như biểu tượng của tình yêu nồng nàn, sự trân trọng và lòng biết ơn sâu sắc. Khi nói đến một món quà tinh tế và đầy ý nghĩa, giỏ hoa hồng đỏ đẹp là sự lựa chọn hoàn hảo để gửi gắm tình cảm chân thành và tạo nên những khoảnh khắc đáng nhớ trong các dịp đặc biệt.
1. Đặc Điểm Nổi Bật Của Giỏ Hoa Hồng Đỏ
Giỏ hoa hồng đỏ không chỉ

DEBUG:scrapy.core.engine:Crawled (200) <GET https://hoatuoimymy.com/gio-hoa-m178/> (referer: None)
DEBUG:scrapy.core.scraper:Scraped from <200 https://hoatuoimymy.com/gio-hoa-m178/>
{'url': 'https://hoatuoimymy.com/gio-hoa-m178/', 'content': 'Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:\n- Hoa tươi mới được nhập về trong ngày\n- Cập nhật xu hướng hoa mới nhất trên thị trường\n- Các thiết kế hoa độc lạ và cực kỳ bắt mắt\n- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận\n- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi\n- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách\n- Hoàn trả tiền khi khách hàng không hài lòng\n- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop\nGiỏ Hoa M178\nGiỏ Hoa – Món Quà Hoàn Hảo Cho Mọi Dịp\nGiỏ hoa luôn là sự lựa chọn hàng đầu khi bạn muốn gửi tặng một món quà ý nghĩa đến người thân, bạn bè, hay đối tác. Với vẻ đẹp tinh tế và sự kết hợp hài hòa của nhiều loài hoa, giỏ ho

Giỏ Hoa M178
====> h1_tag Giỏ Hoa M178
====>description Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:
- Hoa tươi mới được nhập về trong ngày
- Cập nhật xu hướng hoa mới nhất trên thị trường
- Các thiết kế hoa độc lạ và cực kỳ bắt mắt
- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận
- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi
- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách
- Hoàn trả tiền khi khách hàng không hài lòng
- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop
Giỏ Hoa M178
Giỏ Hoa – Món Quà Hoàn Hảo Cho Mọi Dịp
Giỏ hoa luôn là sự lựa chọn hàng đầu khi bạn muốn gửi tặng một món quà ý nghĩa đến người thân, bạn bè, hay đối tác. Với vẻ đẹp tinh tế và sự kết hợp hài hòa của nhiều loài hoa, giỏ hoa không chỉ mang đến sự tươi mới mà còn thể hiện tình cảm sâu sắc của người tặng.
1. Giỏ Hoa – Biểu Tượng Của Sự Tinh Tế
Giỏ hoa là sự kết hợp khéo léo của các loài hoa khác nhau, mỗi loài hoa mang 

...

DEBUG:scrapy.core.engine:Crawled (200) <GET https://hoatuoimymy.com/gio-hoa-m175/> (referer: None)
DEBUG:scrapy.core.scraper:Scraped from <200 https://hoatuoimymy.com/gio-hoa-m175/>
{'url': 'https://hoatuoimymy.com/gio-hoa-m175/', 'content': 'Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:\n- Hoa tươi mới được nhập về trong ngày\n- Cập nhật xu hướng hoa mới nhất trên thị trường\n- Các thiết kế hoa độc lạ và cực kỳ bắt mắt\n- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận\n- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi\n- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách\n- Hoàn trả tiền khi khách hàng không hài lòng\n- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop\nGiỏ Hoa M175\nGiỏ Hoa – Món Quà Hoàn Hảo Cho Mọi Dịp\nGiỏ hoa luôn là sự lựa chọn hàng đầu khi bạn muốn gửi tặng một món quà ý nghĩa đến người thân, bạn bè, hay đối tác. Với vẻ đẹp tinh tế và sự kết hợp hài hòa của nhiều loài hoa, giỏ ho

Giỏ Hoa M175
====> h1_tag Giỏ Hoa M175
====>description Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:
- Hoa tươi mới được nhập về trong ngày
- Cập nhật xu hướng hoa mới nhất trên thị trường
- Các thiết kế hoa độc lạ và cực kỳ bắt mắt
- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận
- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi
- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách
- Hoàn trả tiền khi khách hàng không hài lòng
- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop
Giỏ Hoa M175
Giỏ Hoa – Món Quà Hoàn Hảo Cho Mọi Dịp
Giỏ hoa luôn là sự lựa chọn hàng đầu khi bạn muốn gửi tặng một món quà ý nghĩa đến người thân, bạn bè, hay đối tác. Với vẻ đẹp tinh tế và sự kết hợp hài hòa của nhiều loài hoa, giỏ hoa không chỉ mang đến sự tươi mới mà còn thể hiện tình cảm sâu sắc của người tặng.
1. Giỏ Hoa – Biểu Tượng Của Sự Tinh Tế
Giỏ hoa là sự kết hợp khéo léo của các loài hoa khác nhau, mỗi loài hoa mang 

DEBUG:scrapy.core.engine:Crawled (200) <GET https://hoatuoimymy.com/gio-hoa-m174/> (referer: None)
DEBUG:scrapy.core.scraper:Scraped from <200 https://hoatuoimymy.com/gio-hoa-m174/>
{'url': 'https://hoatuoimymy.com/gio-hoa-m174/', 'content': 'Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:\n- Hoa tươi mới được nhập về trong ngày\n- Cập nhật xu hướng hoa mới nhất trên thị trường\n- Các thiết kế hoa độc lạ và cực kỳ bắt mắt\n- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận\n- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi\n- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách\n- Hoàn trả tiền khi khách hàng không hài lòng\n- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop\nGiỏ Hoa M174\nGiỏ Hoa – Món Quà Hoàn Hảo Cho Mọi Dịp\nGiỏ hoa luôn là sự lựa chọn hàng đầu khi bạn muốn gửi tặng một món quà ý nghĩa đến người thân, bạn bè, hay đối tác. Với vẻ đẹp tinh tế và sự kết hợp hài hòa của nhiều loài hoa, giỏ ho

Giỏ Hoa M174
====> h1_tag Giỏ Hoa M174
====>description Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:
- Hoa tươi mới được nhập về trong ngày
- Cập nhật xu hướng hoa mới nhất trên thị trường
- Các thiết kế hoa độc lạ và cực kỳ bắt mắt
- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận
- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi
- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách
- Hoàn trả tiền khi khách hàng không hài lòng
- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop
Giỏ Hoa M174
Giỏ Hoa – Món Quà Hoàn Hảo Cho Mọi Dịp
Giỏ hoa luôn là sự lựa chọn hàng đầu khi bạn muốn gửi tặng một món quà ý nghĩa đến người thân, bạn bè, hay đối tác. Với vẻ đẹp tinh tế và sự kết hợp hài hòa của nhiều loài hoa, giỏ hoa không chỉ mang đến sự tươi mới mà còn thể hiện tình cảm sâu sắc của người tặng.
1. Giỏ Hoa – Biểu Tượng Của Sự Tinh Tế
Giỏ hoa là sự kết hợp khéo léo của các loài hoa khác nhau, mỗi loài hoa mang 

INFO:scrapy.crawler:Received SIGINT, shutting down gracefully. Send again to force 
2024-09-10 09:30:12 [scrapy.crawler] INFO: Received SIGINT, shutting down gracefully. Send again to force 
INFO:scrapy.core.engine:Closing spider (shutdown)
2024-09-10 09:30:12 [scrapy.core.engine] INFO: Closing spider (shutdown)
INFO:scrapy.crawler:Received SIGINT twice, forcing unclean shutdown
2024-09-10 09:30:14 [scrapy.crawler] INFO: Received SIGINT twice, forcing unclean shutdown


In [ ]:
Giỏ hoa là món quà ý nghĩa, kết hợp tinh tế giữa các loài hoa khác nhau, phù hợp với nhiều dịp đặc biệt, và khi mua tại Hoa Tươi My My, bạn sẽ nhận được sản phẩm chất lượng, đa dạng, cùng dịch vụ giao hàng nhanh chóng và tư vấn tận tình.
1. Giỏ Hoa – Biểu Tượng Của Sự Tinh Tế
Giỏ hoa là sự kết hợp khéo léo của các loài hoa khác nhau, mỗi loài hoa mang một ý nghĩa riêng. Khi được sắp xếp vào một giỏ, chúng tạo nên một tác phẩm nghệ thuật tuyệt đẹp. Một giỏ hoa không chỉ là một bó hoa, mà còn là sự kết hợp hài hòa giữa màu sắc, hương thơm, và kiểu dáng, tạo nên một món quà đầy ý nghĩa.

Giỏ hoa là món quà ý nghĩa, kết hợp tinh tế giữa các loài hoa khác nhau, phù hợp với nhiều dịp đặc biệt, và khi mua tại Hoa Tươi My My, bạn sẽ nhận được sản phẩm chất lượng, đa dạng, cùng dịch vụ giao hàng nhanh chóng và tư vấn tận tình.
2. Những Dịp Thích Hợp Để Tặng Giỏ Hoa
Giỏ hoa là món quà phù hợp cho nhiều dịp khác nhau:
- Sinh nhật:  Một giỏ hoa tươi rực rỡ sẽ là món quà tuyệt vời để chúc mừng sinh nhật người thân yêu. Hoa hồng, hoa lily, và hoa cẩm chướng thường được chọn để gửi gắm lời chúc phúc.
- Khai trương:  Giỏ hoa tặng khai trương thường được chọn với những loài hoa mang ý nghĩa may mắn như hoa đồng tiền, hoa lan, và hoa hướng dương.
- Ngày kỷ niệm:  Dành tặng một giỏ hoa vào dịp kỷ niệm là cách thể hiện tình cảm chân thành và sự tri ân đến người bạn đời hoặc người thân yêu.
- Ngày lễ:  Giỏ hoa cũng là món quà lý tưởng trong các dịp lễ như Valentine, Quốc tế Phụ nữ, Ngày của Mẹ, và Noel.

Giỏ hoa là món quà ý nghĩa, kết hợp tinh tế giữa các loài hoa khác nhau, phù hợp với nhiều dịp đặc biệt, và khi mua tại Hoa Tươi My My, bạn sẽ nhận được sản phẩm chất lượng, đa dạng, cùng dịch vụ giao hàng nhanh chóng và tư vấn tận tình.
3. Cách Lựa Chọn Giỏ Hoa Phù Hợp
Khi chọn giỏ hoa, bạn cần lưu ý đến màu sắc, loại hoa và thông điệp mà bạn muốn truyền tải:
- Màu sắc:  Màu sắc của hoa trong giỏ nên phù hợp với dịp và đối tượng nhận. Chẳng hạn, màu đỏ thường biểu trưng cho tình yêu và sự đam mê, màu trắng thể hiện sự tinh khiết và ngây thơ, trong khi màu vàng tượng trưng cho niềm vui và hạnh phúc.
- Loại hoa:  Mỗi loài hoa đều mang một ý nghĩa riêng. Hoa hồng thể hiện tình yêu, hoa lily biểu tượng cho sự thuần khiết, và hoa hướng dương tượng trưng cho sự tươi mới và năng lượng tích cực.
- Kiểu dáng:  Kiểu dáng của giỏ hoa cũng là yếu tố quan trọng. Một giỏ hoa với thiết kế tinh tế, sang trọng sẽ làm nổi bật ý nghĩa và giá trị của món quà.

Giỏ hoa là món quà ý nghĩa, kết hợp tinh tế giữa các loài hoa khác nhau, phù hợp với nhiều dịp đặc biệt, và khi mua tại Hoa Tươi My My, bạn sẽ nhận được sản phẩm chất lượng, đa dạng, cùng dịch vụ giao hàng nhanh chóng và tư vấn tận tình.
4. Lợi Ích Khi Mua Giỏ Hoa Tại Hoa Tươi My My
Khi bạn đặt giỏ hoa tại Hoa Tươi My My, bạn không chỉ nhận được sản phẩm chất lượng mà còn hưởng nhiều lợi ích vượt trội:
- Đa dạng mẫu mã:  Hoa Tươi My My cung cấp nhiều mẫu giỏ hoa đẹp, từ những thiết kế cổ điển đến hiện đại, phù hợp với nhiều dịp khác nhau.
- Giao hàng nhanh chóng:  Dịch vụ giao hoa nhanh chóng và chuyên nghiệp giúp bạn gửi tặng giỏ hoa đến người nhận đúng thời điểm.
- Tư vấn tận tình:  Đội ngũ nhân viên giàu kinh nghiệm sẵn sàng tư vấn giúp bạn chọn giỏ hoa phù hợp nhất với yêu cầu của mình.
- Giá cả hợp lý:  Hoa Tươi My My cam kết mang đến cho khách hàng những giỏ hoa đẹp với mức giá hợp lý, cạnh tranh.